In [ ]:
import tensorflow as tf
#from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt
import pickle

**Load model and data**

In [ ]:
model = tf.keras.models.load_model("CNN_D0_D4.h5")
model.trainable = False

with open('cifar_img.pkl', 'rb') as file:
    img = pickle.load(file)
with open('cifar_lab.pkl', 'rb') as file:
    lab = pickle.load(file)

**Function of generating adversarial examples**

In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

def create_adv(input_image, input_label):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model(input_image)
        loss = loss_object(input_label, prediction)

    
    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad

**Generate adversarial perturbations**

In [ ]:
label = tf.one_hot(lab.reshape(100), 5)

perturbations = create_adv(tf.convert_to_tensor(img, dtype=float), label)
plt.imshow(perturbations[0] * 0.5 + 0.5);  # To change [-1, 1] to [0,1]

**Report test accuracy**

In [ ]:
epsilons = [0.015, 0.031]

plt.figure()
for i, eps in enumerate(epsilons):
    adv_x = img + eps*perturbations
    adv_x = tf.clip_by_value(adv_x, 0, 1)
    plt.subplot(2,2,2*i+1)
    plt.imshow(img[3])
    prediction_true = model(img)
    prediction = model(adv_x)
    prediction_clean_result = np.mean(lab.reshape(100) == np.argmax(prediction_true, axis=1))
    prediction_adv_result = np.mean(lab.reshape(100) == np.argmax(prediction, axis=1))
    plt.subplot(2,2,2*i+2)
    plt.imshow(adv_x[3])
    print(prediction_clean_result)
    print(prediction_adv_result)